In [ ]:
// Enable Deno Kernel to run this notebook


Notebook looks for enemies sprites in existing https://github.com/freedoom/freedoom commits and saves them in structured way with commit metadata (to identify authors and creation dates)

In [3]:
// Clone the repository to avoid rate limits and increase speed of parsing
const cloneCmd = new Deno.Command("git", {
  args: [
    "clone",
    "--bare",
    "https://github.com/freedoom/freedoom.git",
    "freedoom.git",
  ],
});

cloneCmd.outputSync();


{
  success: false,
  code: 128,
  signal: null,
  stdout: [Getter],
  stderr: [Getter]
}

In [ ]:
import { TextLineStream } from "@std/streams";

const CONFIG = {
  // Path to your local bare repo
  repoPath: "freedoom.git",
  // Base URL for generating links
  githubUrl: "https://github.com/freedoom/freedoom",
  // Regex to match filenames
  fileRegex: /sprites\/POSS/i,
  // Optimization: Tell git to only look in this folder for the snapshot
  searchFolder: "sprites",
  // Output filename
  outputFile: "scan_results.json",
};

/**
 * Helper: Runs git ls-tree to get ALL files existing at a specific commit.
 * This returns the full list of files matching the regex at that point in time.
 */
async function getSnapshotFiles(sha: string): Promise<string[]> {
  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "ls-tree",
      "-r", // Recursive
      "--name-only", // Just filenames
      sha, // The commit hash
      CONFIG.searchFolder, // Optimization: Limit scope
    ],
    stdout: "piped",
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout);

  return output
    .split("\n")
    .map((s) => s.trim())
    .filter((s) => s.length > 0 && CONFIG.fileRegex.test(s));
}

async function main() {
  console.log(`📂 Opening Repo: ${CONFIG.repoPath}`);

  try {
    await Deno.stat(CONFIG.repoPath);
  } catch {
    console.error(`❌ Error: Path not found: ${CONFIG.repoPath}`);
    Deno.exit(1);
  }

  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "log",
      "--name-status",
      "--pretty=format:__COMMIT__|%H|%cd|%an|%s",
      "--date=iso-strict",
      "HEAD",
    ],
    stdout: "piped",
  });

  const process = cmd.spawn();
  console.log("🚀 Git process started. Scanning history...");

  const lines = process.stdout
    .pipeThrough(new TextDecoderStream())
    .pipeThrough(new TextLineStream());

  const results: any[] = [];
  let currentCommit: any = null;
  let commitCount = 0;

  // --- Helper to process a finished commit ---
  const finishCommit = async (commit: any) => {
    // Only process if we found relevant changes (A or M)
    if (commit && commit.changesMap.size > 0) {
      // 1. Fetch snapshot of ALL matching files at this commit
      const allPaths = await getSnapshotFiles(commit.sha);

      // 2. Build the unified 'files' list
      // This merges the change status (A/M) with the full list of existing files
      const mergedFiles = allPaths.map((path) => {
        const status = commit.changesMap.get(path) || "Existing";
        return {
          path: path,
          status: status, // "A", "M", or "Existing"
          url: `${CONFIG.githubUrl}/blob/${commit.sha}/${path}`,
        };
      });

      // 3. Sort: Changed files first, then alphabetical
      mergedFiles.sort((a, b) => {
        const aChanged = a.status !== "Existing";
        const bChanged = b.status !== "Existing";
        // If a is changed and b is not, a comes first
        if (aChanged && !bChanged) return -1;
        if (!aChanged && bChanged) return 1;
        // Otherwise sort by path
        return a.path.localeCompare(b.path);
      });

      // Assign to final property
      commit.files = mergedFiles;

      // Remove the temporary map
      delete commit.changesMap;

      console.log(
        `\r✅ MATCH: ${commit.sha.slice(0, 7)} | ${commit.date} | Changed: ${
          commit.changesMap?.size || mergedFiles.filter((f) =>
            f.status !== "Existing"
          ).length
        } | Total: ${mergedFiles.length}`,
      );

      results.push(commit);
    }
  };

  // --- Stream Loop ---
  for await (const line of lines) {
    const trimmed = line.trim();
    if (!trimmed) continue;

    // 1. Check if line is a New Commit Header
    if (trimmed.startsWith("__COMMIT__|")) {
      await finishCommit(currentCommit);

      commitCount++;
      const parts = trimmed.split("|");
      const sha = parts[1];
      const date = parts[2];
      const author = parts[3];
      const message = parts.slice(4).join("|");

      currentCommit = {
        sha,
        date,
        author,
        message,
        changesMap: new Map<string, string>(), // Temporary storage for changes
        files: [], // Final list will go here
      };

      if (commitCount % 100 === 0) {
        await Deno.stdout.write(
          new TextEncoder().encode(`\r⚡ Scanned ${commitCount} commits...`),
        );
      }
      continue;
    }

    // 2. Check if line is a File Change
    if (currentCommit) {
      const parts = trimmed.split("\t");
      const statusRaw = parts[0];
      const status = statusRaw.charAt(0); // A, M, D...

      // We track A and M. (Deleted files won't show up in ls-tree anyway)
      if (status === "A" || status === "M") {
        const filename = parts[1];
        if (filename && CONFIG.fileRegex.test(filename)) {
          // Store in map: path -> status
          currentCommit.changesMap.set(filename, statusRaw);
        }
      }
    }
  }

  // Finish the very last commit
  await finishCommit(currentCommit);

  // --- Final Output & Save ---
  console.log(`\n\n========================================`);
  console.log(`SCAN COMPLETE`);
  console.log(`Total Commits Scanned: ${commitCount}`);
  console.log(`Matching Commits:      ${results.length}`);
  console.log(`========================================`);

  if (results.length > 0) {
    const jsonOutput = JSON.stringify(results, null, 2);
    await Deno.writeTextFile(CONFIG.outputFile, jsonOutput);

    console.log(`\n💾 Success! Results saved to:`);
    console.log(`👉 ${Deno.cwd()}/${CONFIG.outputFile}`);
  } else {
    console.log("No matching added/modified files found.");
  }
}

main().catch(console.error);


📂 Opening Repo: freedoom.git
🚀 Git process started. Scanning history...


Promise { <pending> }

✅ MATCH: 57246ca | 2023-07-16T23:14:24-07:00 | Changed: 6 | Total: 70
✅ MATCH: e1a73c3 | 2021-03-30T09:59:23-03:00 | Changed: 70 | Total: 70
✅ MATCH: cd21998 | 2020-08-20T04:00:31-03:00 | Changed: 9 | Total: 49
✅ MATCH: 72bb41f | 2020-08-14T00:20:55-03:00 | Changed: 9 | Total: 49
✅ MATCH: 9c6c681 | 2017-07-18T22:26:52-07:00 | Changed: 49 | Total: 49
✅ MATCH: 45d013f | 2014-09-27T20:56:10-04:00 | Changed: 49 | Total: 49
✅ MATCH: 27aca39 | 2006-05-09T16:06:11Z | Changed: 50 | Total: 50


SCAN COMPLETE
Total Commits Scanned: 3223
Matching Commits:      7

💾 Success! Results saved to:
👉 /Users/philipp/Documents/GitHub/freedoom-bestiary/src/sprites-parser/scan_results.json


In [ ]:
// Run with: deno run --allow-read --allow-write --allow-net --allow-run generate_animations_final_fix.ts

import { ensureDir } from "https://deno.land/std@0.220.0/fs/ensure_dir.ts";
import { join } from "https://deno.land/std@0.220.0/path/mod.ts";

const CONFIG = {
  inputFile: "scan_results.json",
  outputDir: "webp",
  tempDir: "temp_frames",
  delay: 40, // 200ms per frame // changed 20 to 40
  concurrency: 5,
};

const SPRITE_REGEX = /sprites\/POSS([a-z])(\d)\.(png|gif)/i;

interface FileEntry {
  path: string;
  status: string;
  url: string;
}

interface CommitEntry {
  sha: string;
  date: string;
  files: FileEntry[];
}

async function checkImageMagick() {
  try {
    const cmd = new Deno.Command("magick", { args: ["-version"] });
    const { success } = await cmd.output();
    if (!success) throw new Error();
  } catch {
    console.error("❌ Error: ImageMagick ('magick') not found.");
    Deno.exit(1);
  }
}

// --- Image Utils ---

async function getMaxDimensions(
  folderPath: string,
): Promise<{ w: number; h: number }> {
  const cmd = new Deno.Command("magick", {
    args: ["identify", "-format", "%w,%h\n", join(folderPath, "*")],
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout).trim();

  let maxW = 0;
  let maxH = 0;

  for (const line of output.split("\n")) {
    const parts = line.split(",");
    if (parts.length < 2) continue;

    const w = parseInt(parts[0], 10);
    const h = parseInt(parts[1], 10);

    if (!isNaN(w) && w > maxW) maxW = w;
    if (!isNaN(h) && h > maxH) maxH = h;
  }

  return { w: maxW, h: maxH };
}

function getRawUrl(blobUrl: string): string {
  return blobUrl
    .replace("github.com", "raw.githubusercontent.com")
    .replace("/blob/", "/");
}

function isValidImageHeader(data: Uint8Array): boolean {
  if (data.length < 4) return false;
  if (data[0] === 0x47 && data[1] === 0x49 && data[2] === 0x46) return true; // GIF
  if (
    data[0] === 0x89 && data[1] === 0x50 && data[2] === 0x4E && data[3] === 0x47
  ) return true; // PNG
  return false;
}

async function downloadRealImage(
  url: string,
  depth = 0,
): Promise<Uint8Array | null> {
  if (depth > 3) return null;
  const res = await fetch(url);
  if (!res.ok) return null;
  const buffer = await res.arrayBuffer();
  const data = new Uint8Array(buffer);
  if (isValidImageHeader(data)) return data;
  try {
    const textContent = new TextDecoder().decode(data).trim();
    if (textContent.length < 500 && !textContent.includes("\0")) {
      const currentUrlObj = new URL(url);
      const targetUrl = new URL(textContent, new URL(".", currentUrlObj)).href;
      if (targetUrl !== url) return downloadRealImage(targetUrl, depth + 1);
    }
  } catch { /* ignore */ }
  return null;
}

function getAnimationSequence(files: FileEntry[]) {
  const frames = new Map<string, FileEntry>();
  for (const file of files) {
    const match = file.path.match(SPRITE_REGEX);
    if (!match) continue;
    const letter = match[1].toUpperCase();
    const angle = parseInt(match[2], 10);
    if (frames.has(letter)) {
      const existing = frames.get(letter)!;
      const existingMatch = existing.path.match(SPRITE_REGEX)!;
      const existingAngle = parseInt(existingMatch[2], 10);
      if (existingAngle !== 1 && angle === 1) frames.set(letter, file);
      else if (existingAngle !== 1 && existingAngle !== 0 && angle === 0) {
        frames.set(letter, file);
      }
    } else {
      frames.set(letter, file);
    }
  }
  return Array.from(frames.keys()).sort().map((key) => frames.get(key)!);
}

// --- Main Processing ---

async function processCommit(commit: CommitEntry) {
  const shortSha = commit.sha.slice(0, 7);
  const sequence = getAnimationSequence(commit.files);

  if (sequence.length === 0) return;

  const commitTempDir = join(CONFIG.tempDir, shortSha);
  await ensureDir(commitTempDir);

  let validFramesCount = 0;

  try {
    let index = 0;
    for (const file of sequence) {
      const rawUrl = getRawUrl(file.url);
      const ext = file.path.split(".").pop() || "png";
      const localName = `${String(index).padStart(3, "0")}.${ext}`;
      const localPath = join(commitTempDir, localName);

      const imageBuffer = await downloadRealImage(rawUrl);
      if (!imageBuffer) continue;

      await Deno.writeFile(localPath, imageBuffer);
      validFramesCount++;
      index++;
    }

    if (validFramesCount < 2) return;

    // 1. Calculate Original Max Dimensions
    const { w, h } = await getMaxDimensions(commitTempDir);

    if (w === 0 || h === 0) {
      console.warn(`⚠️  Could not determine dimensions for ${shortSha}`);
      return;
    }

    const outputFilename = `poss-${shortSha}.webp`;
    const outputPath = join(CONFIG.outputDir, outputFilename);

    // 2. Build ImageMagick Command
    // -sample 200%: Automatically uses Nearest Neighbor (pixel doubling)
    // -define webp:lossless=true: Prevents compression artifacts
    const magickArgs = [
      "-delay",
      String(CONFIG.delay),
      "-dispose",
      "2",
      "-background",
      "none",
      "-loop",
      "0",
      join(commitTempDir, "*"),
      // Remove Cyan
      "-transparent",
      "cyan",
      // Align Bottom-Center
      "+repage",
      "-gravity",
      "South",
      "-extent",
      `${w}x${h}`,
      // Scale 2x Hard Edges (Implies interpolation: Nearest)
      "-sample",
      "200%",
      // Force Lossless WebP
      "-define",
      "webp:lossless=true",
      outputPath,
    ];

    const cmd = new Deno.Command("magick", { args: magickArgs });
    const { success, stderr } = await cmd.output();

    if (!success) {
      throw new Error(new TextDecoder().decode(stderr));
    }

    console.log(
      `✅ Generated: ${outputFilename} (${validFramesCount} frames, Size: ${
        w * 2
      }x${h * 2})`,
    );
  } catch (err) {
    console.error(`❌ Error processing ${shortSha}:`, err.message);
  } finally {
    try {
      await Deno.remove(commitTempDir, { recursive: true });
    } catch { /* ignore */ }
  }
}

async function main() {
  await checkImageMagick();

  console.log(`📂 Reading ${CONFIG.inputFile}...`);
  try {
    const raw = await Deno.readTextFile(CONFIG.inputFile);
    const commits: CommitEntry[] = JSON.parse(raw);

    console.log(`📁 Ensuring output directory: ${CONFIG.outputDir}`);
    await ensureDir(CONFIG.outputDir);
    await ensureDir(CONFIG.tempDir);

    console.log(`🚀 Processing ${commits.length} commits...`);

    for (let i = 0; i < commits.length; i += CONFIG.concurrency) {
      const chunk = commits.slice(i, i + CONFIG.concurrency);
      await Promise.all(chunk.map(processCommit));
    }

    try {
      await Deno.remove(CONFIG.tempDir, { recursive: true });
    } catch { /* ignore */ }

    console.log("\n✨ All Done!");
  } catch (e) {
    console.error("Error:", e.message);
  }
}

main().catch(console.error);


Promise { <pending> }

📂 Reading scan_results.json...
📁 Ensuring output directory: webp_x2
🚀 Processing 7 commits...
✅ Generated: poss-e1a73c3.webp (21 frames, Size: 134x122)
✅ Generated: poss-57246ca.webp (21 frames, Size: 134x122)
✅ Generated: poss-72bb41f.webp (21 frames, Size: 142x128)
✅ Generated: poss-9c6c681.webp (21 frames, Size: 114x124)
✅ Generated: poss-cd21998.webp (21 frames, Size: 134x122)
✅ Generated: poss-27aca39.webp (21 frames, Size: 114x124)
✅ Generated: poss-45d013f.webp (21 frames, Size: 114x124)

✨ All Done!
